In [ ]:
!pip install sentence_transformers pytrec-eval-terrier ir_datasets faiss-gpu cython rank_bm25 python-terrier

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.9/337.9 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.8/111.8 kB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [ ]:
import faiss
import numpy as np
import pyterrier as pt
import ir_datasets
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, CrossEncoder
import logging
import os
from google.colab import drive
from rank_bm25 import BM25Okapi
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
if not pt.started():
    pt.init()

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords, wordnet

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cranfield = ir_datasets.load("cranfield")
model = SentenceTransformer("all-mpnet-base-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def preprocess(text):
    # Remove non-alphanumeric characters (excluding spaces)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text.lower())
    return text

docs = []
for doc in cranfield.docs_iter():
    text = preprocess(doc.text)
    docs.append({ 'doc_id': doc.doc_id, 'text': text})

queries = []
for query in cranfield.queries_iter():
    query_text = preprocess (query.text)
    queries.append({ 'query_id': query.query_id, 'text': query_text})

docs_df = pd.DataFrame(docs)
docs_df.rename(columns={"doc_id": "docno"}, inplace=True)
indexer = pt.DFIndexer("./pre_terrier_index", overwrite=True)
index_ref = indexer.index(docs_df["text"], docs_df["docno"])
index = pt.IndexFactory.of(index_ref)

bm25 = pt.BatchRetrieve(index, wmodel="BM25")
queries_df = pd.DataFrame(queries)
queries_df.rename(columns={"query_id": "qid"}, inplace=True)
queries_df.rename(columns={"text": "query"}, inplace=True)

result = bm25.transform(queries_df)

qrels_df = pd.DataFrame([ {"qid": qrel.query_id, "docno": qrel.doc_id, "label": qrel.relevance} for qrel in cranfield.qrels_iter()])


evaluator = pt.Experiment(
    [bm25],
    queries_df,
    qrels_df,
    eval_metrics=["P_10", "P_5","P_1", "map"],
    names=["BM25"]
)

print(evaluator)

17:42:27.470 [main] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (471) - further warnings are suppressed
17:42:28.701 [main] WARN org.terrier.structures.indexing.Indexer - Indexed 2 empty documents
   name    P_10       P_5       P_1       map
0  BM25  0.0125  0.011842  0.013158  0.012662


In [ ]:
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def preprocess(text):
    # Remove non-alphanumeric characters (excluding spaces)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text.lower())
    tokens = word_tokenize(text)
    tokens = [t for t in tokens if t not in string.punctuation]
    tokens = [t for t in tokens if t not in stop_words]
    tokens = [stemmer.stem(t) for t in tokens]
    return ' '.join(tokens)

processed_docs = []
for doc in cranfield.docs_iter():
    preprocessed_text = preprocess(doc.text)
    processed_docs.append({
        'doc_id': doc.doc_id,
        'text': preprocessed_text
    })

# Preprocess each query in the Cranfield dataset
processed_queries = []
for query in cranfield.queries_iter():
    preprocessed_query_text = preprocess(query.text)
    processed_queries.append({
        'query_id': query.query_id,
        'text': preprocessed_query_text
    })

In [ ]:
docs_df = pd.DataFrame(processed_docs)
docs_df.rename(columns={"doc_id": "docno"}, inplace=True)
indexer = pt.DFIndexer("./terrier_index", overwrite=True)
index_ref = indexer.index(docs_df["text"], docs_df["docno"])
index = pt.IndexFactory.of(index_ref)

bm25 = pt.BatchRetrieve(index, wmodel="BM25")
queries_df = pd.DataFrame(processed_queries)
queries_df.rename(columns={"query_id": "qid"}, inplace=True)
queries_df.rename(columns={"text": "query"}, inplace=True)
result = bm25.transform(queries_df)

qrels_df = pd.DataFrame([ {"qid": qrel.query_id, "docno": qrel.doc_id, "label": qrel.relevance} for qrel in cranfield.qrels_iter()])


17:43:11.387 [main] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (471) - further warnings are suppressed
17:43:13.359 [main] WARN org.terrier.structures.indexing.Indexer - Indexed 2 empty documents


In [ ]:
evaluator = pt.Experiment(
    [bm25],
    queries_df,
    qrels_df,
    eval_metrics=["P_10", "P_5","P_1", "map"],
    names=["BM25"]
)

# Display the evaluation results
print(evaluator)

   name      P_10       P_5       P_1      map
0  BM25  0.010526  0.010526  0.013158  0.01232


In [ ]:
# Paths to the embeddings files
doc_emb_path = './drive/MyDrive/document_embeddings.npy'
query_emb_path = './drive/MyDrive/query_embeddings.npy'

d_i = []
d_e = []
q_i = []
q_e = []

# Check if embeddings files exist and load them; otherwise, create and save
if os.path.exists(doc_emb_path) and os.path.exists(query_emb_path):
    d_e = np.load(doc_emb_path)
    q_e = np.load(query_emb_path)
else:
    # Assume `model` and `cranfield` are already defined and loaded as before
    d_e = []
    for row in tqdm(cranfield.docs_iter()):
        prompt = row.text
        embedding = model.encode(prompt)
        d_e.append(embedding)
    d_e = np.array(d_e)
    np.save(doc_emb_path, d_e)

    q_e = []
    for row in tqdm(cranfield.queries_iter()):
        prompt = row.text
        embedding = model.encode(prompt)
        q_e.append(embedding)
    q_e = np.array(q_e)
    np.save(query_emb_path, q_e)

for row in tqdm(cranfield.docs_iter()):
    d_i.append(int(row.doc_id))
for row in tqdm(cranfield.queries_iter()):
    q_i.append(int(row.query_id))

1400it [00:00, 32328.36it/s]
225it [00:00, 59078.40it/s]


In [ ]:
# Define evaluation metrics
def precision_at_k(y_true, y_pred, k):
    relevant_items = 0
    for i in range(min(k, len(y_pred))):
        if y_pred[i] in y_true:
            relevant_items += 1
    return relevant_items / k

def average_precision(y_true, y_pred):
    score = 0.0
    num_hits = 0
    for i, p in enumerate(y_pred):
        if p in y_true:
            num_hits += 1
            score += num_hits / (i + 1)
    return score / len(y_true)

def mean_average_precision(y_true_list, y_pred_list):
    return np.mean([average_precision(y_true, y_pred) for y_true, y_pred in zip(y_true_list, y_pred_list)])


In [ ]:
# Create a FAISS index for L2 distance
faiss_index = faiss.IndexFlatL2(d_e.shape[1])
faiss_index.add(d_e)
# Perform the search KNN
D, I = faiss_index.search(q_e, 10)
# Retrieve rankings based on document IDs
rankings = np.array(d_i)[I]

In [ ]:
# Retrieve rankings based on document IDs
document_ids = [doc.doc_id for doc in cranfield.docs_iter()]
rankings_before = [[document_ids[idx] for idx in indices] for indices in I]


# Calculate P@10 and MAP before re-ranking
p_10_before = np.mean([precision_at_k(y_true, y_pred, 10) for y_true, y_pred in zip(list(cranfield.qrels_dict().values()), rankings_before)])
p_5_before = np.mean([precision_at_k(y_true, y_pred, 5) for y_true, y_pred in zip(list(cranfield.qrels_dict().values()), rankings_before)])
p_1_before = np.mean([precision_at_k(y_true, y_pred, 1) for y_true, y_pred in zip(list(cranfield.qrels_dict().values()), rankings_before)])
map_before = mean_average_precision(list(cranfield.qrels_dict().values()), rankings_before)

print("P@10 Before Re-ranking:", p_10_before)
print("P@5 Before Re-ranking:", p_5_before)
print("P@1 Before Re-ranking:", p_1_before)
print("MAP Before Re-ranking:", map_before)

P@10 Before Re-ranking: 0.32355555555555554
P@5 Before Re-ranking: 0.4524444444444445
P@1 Before Re-ranking: 0.7244444444444444
MAP Before Re-ranking: 0.3648032487862154


In [ ]:
# Load CrossEncoder
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
def rerank_with_cross_encoder(query_text, top_docs_texts, top_doc_ids):
    pairs = [[query_text, doc_text] for doc_text in top_docs_texts]
    scores = cross_encoder.predict(pairs, show_progress_bar=True)
    ranked_pairs = sorted(zip(scores, top_doc_ids), reverse=True, key=lambda x: x[0])
    return [doc_id for _, doc_id in ranked_pairs[:5]]

In [ ]:
document_texts = {doc.doc_id: doc.text for doc in cranfield.docs_iter()}

In [ ]:
# Convert query IDs to strings when creating the dictionary, assuming they are integers in the dataset
query_texts = {str(qry.query_id): qry.text for qry in cranfield.queries_iter()}

# Apply re-ranking for each query
top_5_rankings = []
for query_idx, doc_indices in enumerate(I):
    query_id = str(q_i[query_idx])  # Convert query ID to string
    if query_id in query_texts:
        query_text = query_texts[query_id]  # Get the text of the current query
        top_docs_texts = [document_texts[str(d_i[idx])] for idx in doc_indices if str(d_i[idx]) in document_texts]  # Get the texts of the top 10 documents
        top_doc_ids = [str(d_i[idx]) for idx in doc_indices if str(d_i[idx]) in document_texts]  # Get the IDs of the top 10 documents
        top_5_ids = rerank_with_cross_encoder(query_text, top_docs_texts, top_doc_ids)
        top_5_rankings.append(top_5_ids)
    else:
        print(f"Query ID {query_id} not found in query_texts")


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# Calculate P@10 and MAP before re-ranking
p_1_after = np.mean([precision_at_k(y_true, y_pred, 1) for y_true, y_pred in zip(list(cranfield.qrels_dict().values()), top_5_rankings)])
p_5_after = np.mean([precision_at_k(y_true, y_pred, 5) for y_true, y_pred in zip(list(cranfield.qrels_dict().values()), top_5_rankings)])
map_after = mean_average_precision(list(cranfield.qrels_dict().values()), top_5_rankings)
# Print the metrics
print("P@1 After Re-ranking:", p_1_after)
print("P@5 After Re-ranking:", p_5_after)
print("MAP After Re-ranking:", map_after)


P@1 After Re-ranking: 0.7644444444444445
P@5 After Re-ranking: 0.4720000000000001
MAP After Re-ranking: 0.31601826600139654


In [ ]:
def retrieve_and_rerank_documents(query, k=10):
    # Encode the query to get its embedding
    query_embedding = model.encode(query)
    D, I = index.search(np.array([query_embedding]), k)  # Perform search for top k similar documents

    # Retrieve initial documents based on the indices found
    doc_lookup = list(cranfield.docs_iter())  # Assuming the dataset is not too large to fit in memory
    initial_retrieved_docs = []
    for idx in I.flatten():
        doc_id = doc_lookup[idx].doc_id  # Access document ID
        doc_text = doc_lookup[idx].text  # Access document text
        initial_retrieved_docs.append((doc_id, doc_text))

    # Print initial retrieved documents
    print("Initial Top 10 Documents:")
    for doc in initial_retrieved_docs:
        print(f"Doc ID: {doc[0]}")  # Print the first 150 characters of the document

    # Re-rank documents using the cross-encoder
    pairs = [[query, doc[1]] for doc in initial_retrieved_docs]
    scores = cross_encoder.predict(pairs)
    reranked_docs = sorted(zip(scores, initial_retrieved_docs), reverse=True, key=lambda x: x[0])
    top_5_docs = [doc for _, doc in reranked_docs[:5]]

    # Print re-ranked documents
    print("\nTop 5 Re-Ranked Documents:")
    for doc in top_5_docs:
        print(f"Doc ID: {doc[0]}")  # Print the first 150 characters of the document

    return top_5_docs

In [ ]:
example_query = "global warming"
retrieve_and_rerank_documents(example_query)

Initial Top 10 Documents:
Doc ID: 471
Doc ID: 995
Doc ID: 582
Doc ID: 396
Doc ID: 1345
Doc ID: 620
Doc ID: 480
Doc ID: 621
Doc ID: 1147
Doc ID: 1344

Top 5 Re-Ranked Documents:
Doc ID: 1147
Doc ID: 471
Doc ID: 995
Doc ID: 620
Doc ID: 621


[('1147',
  'heat transfer to bodies traveling at high speed in the upper\natmosphere .\na general method has been developed, using the methods of kinetic\ntheory, whereby the surface temperatures of bodies can be calculated for\nsteady flight at any speed in a rarefied gas .  the particular solution\nwas made for a flat plate., however, the calculations can be easily\nextended to bodies of arbitrary shape .\nit was found that the aerodynamic heating problem in the absence of\nsolar radiation, that is, for the case of nocturnal flight, becomes of\nnegligible importance at altitudes of 125 miles and higher and up to\nsteady flight speeds of 36,000 feet per second .  the effect of solar\nradiation, for the case of daytime flight, becomes increasingly\nimportant as the flight altitude is increased .  at an altitude of 150\nmiles and higher, solar radiation is the predominating factor that\ndetermines skin temperature .  owing to the strong effect of solar\nradiation on skin temperatures a